通过历史数据评估阳床树脂再生能力。

因所填表格不满足数据库第一范式，所以需要处理数据的大量工作。

从更换树脂当天至未来的某一天，当原树脂洗床所多消耗酸的成本可以覆盖更换树脂的成本时，就值得更换树脂。

In [ ]:
#!/usr/bin/python3                                            
# -*- coding:utf-8 -*-    

import pandas as pd
import calendar
import re

# 遍历路径文件
for j in range(1,13,1):
    file_name = '干熄焦生产记录台账/2021/干熄焦生产记录台账（' + str(j) + '月）.xlsx'    
    file_date_info = re.findall(r'\d{1,}', file_name) # 提取路径文件信息
    month = file_date_info[1]
    year = file_date_info[0]

    # 返回（当月总周数，当月总天数）
    month_range = calendar.monthrange(int(year),int(month))

    # 遍历表单
    for i in range(1,month_range[1],1):  # 从第1天开始到当月最后一天
        ws_data = pd.read_excel(file_name, sheet_name = str(i)+"日")
        ws_data_regeneration = year + "-" + month + "-" + str(i) + ws_data.iloc[26:27,[2,5,8]] # sheet包含洗床信息的cell
        ws_data_concat = ws_data_regeneration.to_csv("ws_regeneration.csv",mode='a',index=False, header=False) # 整合当天洗床时间，追加插入文件

In [266]:
regeneration = pd.read_csv(r'ws_regeneration.csv')
regeneration.columns = ["夜","白","中"]
regeneration

,夜,白,中
0,2021-1-2清洗3#、4#、5#过滤器,2021-1-2清洗4#过滤器,2021-1-2清洗1#、3#过滤器\n16:27再生2#阴床，进碱浓度：3.0% 3.0%...
1,2021-1-3清洗4#、5#过滤器,NaN,2021-1-3清洗1#、3#过滤器 ...
2,2021-1-4 ...,2021-1-412:26再生1#阴床，进碱浓度：3.2% 3.0% \n14:30分...,2021-1-416:00分再生2#阳床，进酸浓度：3.0% 3.0%
3,2021-1-5清洗1#、3#过滤器\n1：00分中和排水 （PH：1# 8.5 2# 8...,2021-1-5清洗4#、5#过滤器,2021-1-5清洗1#、3#过滤器 ...
4,2021-1-6清洗1#、3#过滤器 ...,2021-1-6清洗4#、5#过滤器 ...,2021-1-6清洗3#、4#过滤器 ...
...,...,...,...
347,2021-12-2600:11分再生1#阳床，进酸浓度：3.0%，3.0% ...,2021-12-2610:10分再生3#阳床，进酸浓度：3.0%，3.0%,2021-12-2619:11分再生1#阴床，进碱浓度：3.0%，3.0% ...
348,NaN,NaN,2021-12-2716:11分再生1#阴床，进碱浓度：3.0%，3.0% ...
349,2021-12-284:44分再生2#阳床，进酸浓度：3.0%，3.0%,2021-12-288:00分再生2#阴床，进碱浓度：3.1%，3.0% \n10:00...,NaN
350,2021-12-2902：33分再生:1#阴床，进碱浓度：3.1%，3.0% \n04:...,2021-12-29\n8：32分再生3#阴床，进碱浓度：3.1%，3.1% \n11：00...,2021-12-2920：36分再生3#阳床，进酸浓度：3.0%，3.0% 23：0...


测试清洗单个单元格

In [10]:
import pandas as pd
test = pd.read_excel("干熄焦生产记录台账/2020/干熄焦生产记录台账（3月）.xlsx")
ws_regeneration_cell = test.iloc[26:27,[2,5,8]]
ws_regeneration_cell

,Unnamed: 2,Unnamed: 5,Unnamed: 8
26,NaN,NaN,NaN


In [267]:
yang1 = re.compile(r'\d{1,}:\d{1,}分再生2#阳床')
f1yang = re.search(yang1, regeneration[0])
print(f1yang)

KeyError: 0